In [19]:
%config IPCompleter.greedy=True
from sklearn.feature_extraction.text import CountVectorizer
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
import json

In [20]:
#load vocabulary
with open('data/vocabulary.json', 'r') as inf:
    vocabulary = json.load(inf) 
vectorizer = CountVectorizer(vocabulary=vocabulary, token_pattern=r'\S+', stop_words=['<nl>'])

analyzer = vectorizer.build_analyzer()

In [22]:
candidates = []
with open('data/generated.msg', 'r') as inf:
    for msg in inf:
        cur_candidate = analyzer(msg)
        candidates.append(cur_candidate)
        
references = []
with open('data/test.msg', 'r') as inf:
    for msg in inf:
        cur_reference = analyzer(msg)
        references.append([cur_reference])
        
corpus_bleu(references, candidates)

0.04545344475308948

In [37]:
candidates = []
with open('data/generated.msg', 'r') as inf:
    for msg in inf:
        cur_candidate = analyzer(msg)
        candidates.append(cur_candidate)
        
references = []
with open('data/test.msg', 'r') as inf:
    for msg in inf:
        cur_reference = analyzer(msg)
        references.append([cur_reference])
        
n = len(candidates)
bleu_sum = 0
print(references[0], candidates[0])
cnt = 0
for i in range(n):
    bleu_score = sentence_bleu(references[i], candidates[i])
    if (bleu_score > 0.1):
        cnt += 1
        print(' '.join(references[i][0]))
        print(' '.join(candidates[i]))
        print()
    #print(bleu_score)
print('{} good of {} total ({:.0f}%)'.format(cnt, n, cnt * 100 / n))

[['[', 'projects', ']', 'upgrades', 'asm', 'library', 'to', '7', '.', '0', '(', 'final', ')']] ['gui', '-', '88', 'too', 'many', 'events', 'posted', 'on', 'component', 'search', '(', 'continue', ')']
[ gui - test ] newprojectdialogmodel : make explicit click on project location
[ gui - test ] workaround for gui - 117

[ gui - test ] extendedjtreepathfixture : printmodel should output shown values
[ gui - test ] reformat

[ gui - test ] projectstructuredialogmodel . checklibrarypresent : added click on checked library
[ gui - test ] minor improvements

[ vcs - log ] suppress " unused " warning
[ vcs - log ] schedule new indexing requests in case of exception

[ vcs - log ] cleanup
[ vcs - log ] reformat

[ gui - test ] search editor containing necessary text ( gui - 181 )
[ gui - test ] make editor be a function with a lambda receiver

[ file - history ] reorder actions in file history context menu
idea - 182879 vcs : reorder actions in file history popup menu

idea - 201014 maven - com

[ gui - test ] fix transient fatal errors gui - 76

expressionutils # isreferenceto : do not resolve if qualified reference is checked against local / parameter
expressionutils # isreferenceto : return false if variable is null ( like contract suggests ) , fixes ea - 101613 .

find in path : start search when ui is shown ( otherwise it stops immediately )
find in path : stop continuously self - restarting search

[ vcs - log ] use case - sensitive hashing strategy when calculating diff preview revision titles
[ vcs - log ] minor : remove unused field

[ vcs - log ] provide vcslogdata in context
[ vcs - log ] advertise idea - 140935 with a tooltip

[ file - history ] rename comparerevisionsfromhistoryactionprovider to kotlin
[ file - history ] rename filehistoryrefiner to kotlin

[ gui - test ] add wait function to timeouts
[ gui - test ]

[ gui - test ] fix plugins test case
[ gui - test ] filter invisible dialogs

[ vcs - log ] preserve case of indexed paths
[ vcs - log ] remove dupli

git tests : setup username inside the submodule dir
git tests : setup default username for auxiliary repositories

[ vcs - log ] replace assert with logger . asserttrue
[ vcs - log ] diagnostic for ea - 83463

idea - cr - 36771 followup
offline inspection result view : the code simplified as it was suggested in idea - cr - 11873

[ gui - test ] add settings dialog finder
[ gui - test ] add ability to wait closing of a dialog on demand

java mad testing : don ' t expect that " surround with try - with - resources block " preserves compilability
java mad testing : " make * default " intention can break compilability

[ vcs - log ] move vcslogindexservice to the intellij . platform . vcs . log
[ vcs - log ] rename vcsuserregistryimpl to kotlin

updating versions of project dependencies : jdkbuild - > u152b1293 . 12
updating versions of project dependencies : jdkbuild - > u152b1239 . 1

[ vcs - log ] improve git . log . fix . merge . commits . parents . order property description
[ vcs - l

[ gui - test ] fixed stackoverflow error in comboboxfixture
[ gui - test ] react - app / react - native fix

[ gui - test ] added predicate . startwith
[ gui - test ] add bower test

[ gui - test ] kotlin . gui . test . configure . gradle : m * release are not final ones
[ gui - test ] corrected filling of isfinalrelease property

[ gui - test ] recognize check box as a text component
[ gui - test ]

[ gui - test ] added model for environment variables dialog
[ gui - test ] typeaheadtest is catching idea - 180346 problem

[ gui - test ] comboboxfixture : cosmetic changes
[ gui - test ] add support for select by index in comboboxfixture # selectitem

[ gui - test ] runconfigurationmodel : corrected function printhierarchy
[ gui - test ] removed extra level of teardown . now in case of failure ide is closed faster .

[ gui - test ] runconfigurationmodel : added possibility to compare by predicate .
[ gui - test ] added checks of fields in run configuration

[ gui - test ] extendedjcombob

[ vcs - log ] minor : move troveutil to util package

update jediterm - pty to v2 . 10 to fix idea - 187084
update jediterm - pty to v2 . 9 ( idea - 186170 , idea - 118871 , idea - 126617 )

[ file - history ] add @ notnull , remove finals
[ file - history ] rearrange fields

[ file - history ] call refresher in invokelater
[ file - history ] inline showtree method

[ file - history ] revalidate and repaint splitter after new data arrival
[ file - history ] inline getvirtualfile method

[ file - history ] use revision number for getting a lock
[ file - history ] better work with backgroundableactionlock

revert : idea project code style : align multiline chained calls
idea project code style : align multiline chained calls

ui : cosmetic improvements in ' configure transitive dependencies ' dialog ( idea - 195170 )
ui : fix resizing of ' configure transitive dependencies ' dialog ( idea - 193045 )

[ vcs - log ] redirect dvcsutil . getshorthash to vcslogutil . getshorthash
[ vcs - log 

In [35]:
candidates = []
with open('nngen_data/generated.msg', 'r') as inf:
    for msg in inf:
        cur_candidate = analyzer(msg)
        candidates.append(cur_candidate)
        
references = []
with open('nngen_data/test.msg', 'r') as inf:
    for msg in inf:
        cur_reference = analyzer(msg)
        references.append([cur_reference])
        
n = len(candidates)
bleu_sum = 0
print(references[0], candidates[0])
cnt = 0
for i in range(n):
    bleu_score = sentence_bleu(references[i], candidates[i])
    if (bleu_score > 0.1):
        cnt += 1
        print(' '.join(references[i][0]))
        print(' '.join(candidates[i]))
        print()
    #print(bleu_score)
print('{} good of {} total ({:.0f}%)'.format(cnt, n, cnt * 100 / n))

[['fix', 'snapshot', 'version']] ['add', 'demo', 'image']
ignore update ' modules / apps / foundation / portal / .
ignore update ' modules / apps / foundation / portal / .

bump engine . io - client
bump engine . io - client

ignore update ' modules / apps / web - experience / iframe / .
ignore update ' modules / apps / web - experience / iframe / .

prepare release checkstyle - 7 . 1 . 1
prepare release checkstyle - 7 . 1

update h2o - flow version to 0 . 5 . 0 with . . . ( # 684 )
update flow to version 0 . 5 . 1

removed non - needed imports
removed non - needed imports

setting version to 1 . 0 . 133 - snapshot
setting version to 1 . 0 . 71

ignore update ' modules / apps / foundation / roles / .
ignore update ' modules / apps / foundation / roles / .

ignore update ' modules / apps / foundation / petra / .
ignore update ' modules / apps / foundation / petra / .

ignore update ' modules / apps / foundation / portal - cache / .
ignore update ' modules / apps / foundation / portal - 

/Users/ne0n/miniconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/ne0n/miniconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/ne0n/miniconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoo

ignore update ' modules / apps / foundation / frontend - taglib / .

prepare version 23 - beta4 .
prepare version 23 - beta5 .

bump maven lib version to 1 . 0 . 1 .
bump maven lib version to 1 . 0 . 1 .

updated gradle plugin to 2 . 1 . 2 .
update gradle to 2 . 1 . 2

ignore update ' modules / apps / foundation / frontend - theme / .
ignore update ' modules / apps / foundation / frontend - theme / .

ignore update ' modules / apps / foundation / server / .
ignore update ' modules / apps / foundation / server / .

ignore update ' modules / apps / web - experience / iframe / .
ignore update ' modules / apps / web - experience / iframe / .

push version to 15 . 0 . 4 , 15006 .
push version to 15 . 0 . 5 , 15007 .

ignore update ' modules / apps / web - experience / asset / .
ignore update ' modules / apps / web - experience / asset / .

ignore update ' modules / apps / collaboration / flags / .
ignore update ' modules / apps / collaboration / flags / .

restored j2objc - framework depend

ignore update ' modules / apps / collaboration / wiki / .

added translation using weblate ( nepali )
added translation using weblate ( nepali )

ignore update ' modules / apps / foundation / portal - store / .
ignore update ' modules / apps / foundation / portal - store / .

ignore update ' modules / apps / collaboration / document - library / .
ignore update ' modules / apps / collaboration / document - library / .

lps - 60203 - updating alloy 3 . 0 . x to 649200da9388bc851e43a107e3ec2b6db1c994d0
lps - 55259 - updating alloy 3 . 0 . x to 6eb8209f0cdb27ecee0494e1653220159d9d7439

ignore update ' modules / apps / foundation / portal - remote / .
ignore update ' modules / apps / foundation / portal - remote / .

ignore update ' modules / apps / foundation / portal - search / .
ignore update ' modules / apps / foundation / portal - search / .

set revision 3 . 3 . 5 .
set revision 3 . 3 . 5 .

ignore update ' modules / apps / collaboration / microblogs / .
ignore update ' modules / apps